In [1]:
! pip install --quiet environs cyksuid toolz psycopg2-binary typing_json backoff xxhash pyyaml geopandas facebook_business

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
from environs import Env
from adopt.malaria import window
from adopt.malaria import get_conf, get_df, lookalike, get_ad_token
from adopt.facebook.state import CampaignState
from adopt.responses import get_ad_token

env = Env()
env.read_env('.env-upswell-vlab', override=True)

cnf = get_conf(env)
token = get_ad_token(cnf["survey_user"], cnf["chatbase"])
state = CampaignState(env, token, window(4))

In [ ]:
state.adsets

In [29]:
[len(a['targeting']['excluded_geo_locations']['custom_locations']) for a in state.adsets]

[49, 151, 200, 200, 151, 23, 26]

In [ ]:
[len(a['targeting']['geo_locations']['regions']) for a in state.adsets]

In [31]:
from facebook_business.adobjects.targetinggeolocationcity import TargetingGeoLocationCity
import pandas as pd
from adopt.malaria import load_cities
from adopt.marketing import *

def get_control_cities():
    treatment_assignment = pd.read_csv('outs/ma-with-treatment.csv')[['disthash', 'treatment']]
    cities = load_cities('output/cities.csv')

    controls = cities \
        .merge(treatment_assignment) \
        .pipe(lambda df: df[df.treatment == 0]) \
        .pipe(lambda df: df.assign(rad = df.rad + 1))

    return controls

def lookup_region(state):
    regions = {
        'Uttar Pradesh': {
            "country": "IN",
            "key": "1754",
            "name": "Uttar Pradesh"
        },
        'Jharkhand': {
            "country": "IN",
            "key": "1756",
            "name": "Jharkhand"
        },
        'Chhatisgarh': {
            "country": "IN",
            "key": "1755",
            "name": "Chhattisgarh"
        }
    }

    regions = {**regions, 'combined': [v for v in regions.values()]}

    return regions[state]


def create_city(t):
    return {
        TargetingGeoLocationCity.Field.key: t['key'],
        TargetingGeoLocationCity.Field.radius: 25.0,
        TargetingGeoLocationCity.Field.distance_unit: 'kilometer'
    }

def template_targeting(c: AdsetConf, target_key, targets, excluded_cities=[]):
    targeting = {}

    excluded_locs = [create_location(lat, lng, rad)
                     for lat, lng, rad in c.excluded_locs]

    targeting[Targeting.Field.excluded_geo_locations] = {
        TargetingGeoLocation.Field.location_types: ['home'],
        TargetingGeoLocation.Field.custom_locations: excluded_locs,
        TargetingGeoLocation.Field.cities: excluded_cities
    }

    # if targets is None:
    #     key = 
    #     locs = lookup_region(c.cluster.name)

    # else: 
    #     key = 
    #     locs = [create_city(t) for t in targets]
        

    targeting[Targeting.Field.geo_locations] = {
        TargetingGeoLocation.Field.location_types: ['home'],
        target_key: targets
    }

    return targeting

def make_template_adsets(env, state, base_name, controls, targets, target_key, excluded_cities=[]):

    locs = [Location(r.lat, r.lng, r.rad) for _, r in controls.iterrows()]
    cluster = Cluster(base_name, base_name)

    conf = AdsetConf(state.campaign, cluster, None, locs, [], 
                       10000, 'PAUSED', env('FACEBOOK_INSTA_ID'), 
                       48, None, None, None)

    instructions = create_adset(base_name, conf, template_targeting(conf, target_key, targets, excluded_cities))


    return instructions

ImportError: cannot import name 'load_cities' from 'adopt.malaria' (/home/jupyter/work/adopt/adopt/malaria.py)

In [ ]:
from adopt.responses import query

cnf = {'db': 'chatroach', 'user': 'chatroach', 'port': 5432, 'host': '0.0.0.0', 'password': None}




In [41]:
m = treatment_assignment.merge(cities)
m = m[m.state != 'Chhatisgarh']

m = m[['disthash', 'treatment']].drop_duplicates().reset_index()

In [ ]:
import psycopg2

for i, r in m.iterrows():
    with psycopg2.connect(dbname=cnf['db'],
                          user=cnf['user'],
                          host=cnf['host'],
                          port=cnf['port'],
                          password=cnf['password']) as conn:
        with conn.cursor() as cur:
            vals = (r.disthash, bool(r.treatment))
            q = 'insert into clusters(clusterid, treatment) values(%s, %s)'
            cur.execute(q, vals)

In [ ]:
cities = load_cities('output/cities.csv')
treatment_assignment = pd.read_csv('outs/ma-with-treatment.csv')[['disthash', 'treatment']]
cities = cities.merge(treatment_assignment)

districts = cities.groupby('distname') \
                  .head(1) \
                  .reset_index(drop=True)

controls = get_control_cities()

In [ ]:
controls[controls.state == 'Chhatisgarh']

In [ ]:
from facebook_business.adobjects.targeting import Targeting


dir(Targeting.Field)

In [39]:
res = TargetingSearch.search({'type': 'adTargetingCategory', 'country_code': 'LB', 'class': 'demographics'})

In [42]:
res

[<TargetingSearch> {
     "audience_size": 18063462,
     "description": "People who have been married for less than 1 year",
     "id": "6002714398172",
     "name": "Newlywed (1 year)",
     "path": [
         "Newlywed (1 year)"
     ],
     "real_time_cluster": false,
     "type": "life_events"
 },
 <TargetingSearch> {
     "audience_size": 287925012,
     "description": "People who are likely to be parents",
     "id": "6002714398372",
     "name": "Parents (All)",
     "path": [
         "Parents (All)"
     ],
     "real_time_cluster": false,
     "type": "family_statuses"
 },
 <TargetingSearch> {
     "audience_size": 3705965,
     "description": "People who have been engaged for less than 6 months",
     "id": "6002714398772",
     "name": "Newly-engaged (6 months)",
     "path": [
         "Newly-engaged (6 months)"
     ],
     "real_time_cluster": false,
     "type": "life_events"
 },
 <TargetingSearch> {
     "audience_size": 73519660,
     "description": "People who are g

In [ ]:
from facebook_business.adobjects.targetingsearch import TargetingSearch

with open('outs/cities.txt') as f:
    cities = [l.strip() for l in f]

city_locs = [(city, TargetingSearch.search({'type': 'adgeolocation', 'q': city, 'location_types': ['city'], 'country_code': 'IN'}))
             for city in cities]
city_cities = [l[0] for name, l in city_locs]

In [ ]:
city_locs

In [85]:
others = [{'key': '1724', 'name': 'Andhra Pradesh', 'country': 'IN'},
          {'key': '1725', 'name': 'Assam', 'country': 'IN'},
          {'key': '1729', 'name': 'Gujarat', 'country': 'IN'},
          {'key': '1735', 'name': 'Maharashtra', 'country': 'IN'},
          {'key': '1736', 'name': 'Manipur', 'country': 'IN'},
          {'key': '1737', 'name': 'Meghalaya', 'country': 'IN'},
          {'key': '1738', 'name': 'Karnataka', 'country': 'IN'},
          {'key': '1739', 'name': 'Nagaland', 'country': 'IN'},
          {'key': '1743', 'name': 'Rajasthan', 'country': 'IN'},
          {'key': '1744', 'name': 'Tamil Nadu', 'country': 'IN'},
          {'key': '1745', 'name': 'Tripura', 'country': 'IN'},
          {'key': '1746', 'name': 'West Bengal', 'country': 'IN'},
          {'key': '1747', 'name': 'Sikkim', 'country': 'IN'},
          {'key': '1748', 'name': 'Arunachal Pradesh', 'country': 'IN'},
          {'key': '1749', 'name': 'Mizoram', 'country': 'IN'},
          {'key': '1752', 'name': 'Bihar', 'country': 'IN'},
          {'key': '1753', 'name': 'Madhya Pradesh', 'country': 'IN'},
          {'key': '1755', 'name': 'Chhattisgarh', 'country': 'IN'},
          {'key': '1756', 'name': 'Jharkhand', 'country': 'IN'}]

dats = {
    'UP': {
        'controls': controls[controls.state == 'Uttar Pradesh'],
        'targets': [{"country": "IN", "key": "1754", "name": "Uttar Pradesh"}],
        'excluded_cities': [c for c in city_cities if c['region'] == 'Uttar Pradesh' 
                            and c['name'] not in ['Meerut', 'Varanasi']], # are in the cities already
        'target_key': TargetingGeoLocation.Field.regions,
    },
    'NOT-UP': {
        'controls': controls[controls.state != 'Uttar Pradesh'],
        'targets': others,
        'excluded_cities': [c for c in city_cities if c['region'] != 'Uttar Pradesh'],
        'target_key': TargetingGeoLocation.Field.regions,
    }
}

In [ ]:
from adopt.facebook.update import GraphUpdater

instruction = make_template_adsets(env, 
                                   state, 
                                   'ie-rural-template-not-UP', 
                                   **dats['NOT-UP'])

updater = GraphUpdater(state)
report = updater.execute(instruction)
print(report)

In [94]:

 # 'country',
 # 'distance_unit',
 # 'key',
 # 'name',
 # 'radius',
 # 'region',
 # 'region_id'




{'key': '1035921', 'radius': 25.0, 'distance_unit': 'kilometer'}

In [109]:
import geopandas as gpd

gdf = gpd.read_file('outs/clusters.shp').drop(columns=['treatment'])

gdf = gdf.drop(columns=['st_areasha']).merge(districts[['distcode', 'treatment']], how='left')

In [116]:
gdf.to_file('outs/clusters.shp')